<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preamble" data-toc-modified-id="Preamble-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preamble</a></span></li><li><span><a href="#Loading-in-AWAP" data-toc-modified-id="Loading-in-AWAP-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Loading in AWAP</a></span></li><li><span><a href="#Applying--a-Mask" data-toc-modified-id="Applying--a-Mask-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Applying  a Mask</a></span></li><li><span><a href="#Just-North-and-Raindays" data-toc-modified-id="Just-North-and-Raindays-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Just North and Raindays</a></span></li><li><span><a href="#The-RMM" data-toc-modified-id="The-RMM-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>The RMM</a></span><ul class="toc-item"><li><span><a href="#Reading-In" data-toc-modified-id="Reading-In-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Reading In</a></span></li><li><span><a href="#Adding-to-AWAP-and-Removing-Errors" data-toc-modified-id="Adding-to-AWAP-and-Removing-Errors-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Adding to AWAP and Removing Errors</a></span></li></ul></li><li><span><a href="#Percentiles" data-toc-modified-id="Percentiles-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Percentiles</a></span><ul class="toc-item"><li><span><a href="#Climatology" data-toc-modified-id="Climatology-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Climatology</a></span><ul class="toc-item"><li><span><a href="#90th" data-toc-modified-id="90th-6.1.1"><span class="toc-item-num">6.1.1&nbsp;&nbsp;</span>90th</a></span></li><li><span><a href="#95th" data-toc-modified-id="95th-6.1.2"><span class="toc-item-num">6.1.2&nbsp;&nbsp;</span>95th</a></span></li></ul></li><li><span><a href="#AWAP" data-toc-modified-id="AWAP-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>AWAP</a></span><ul class="toc-item"><li><span><a href="#Functions" data-toc-modified-id="Functions-6.2.1"><span class="toc-item-num">6.2.1&nbsp;&nbsp;</span>Functions</a></span></li><li><span><a href="#P90" data-toc-modified-id="P90-6.2.2"><span class="toc-item-num">6.2.2&nbsp;&nbsp;</span>P90</a></span></li><li><span><a href="#P95" data-toc-modified-id="P95-6.2.3"><span class="toc-item-num">6.2.3&nbsp;&nbsp;</span>P95</a></span></li><li><span><a href="#Manual" data-toc-modified-id="Manual-6.2.4"><span class="toc-item-num">6.2.4&nbsp;&nbsp;</span>Manual</a></span></li><li><span><a href="#Exporting" data-toc-modified-id="Exporting-6.2.5"><span class="toc-item-num">6.2.5&nbsp;&nbsp;</span>Exporting</a></span></li></ul></li></ul></li></ul></div>

# Preamble

In [1]:
import xarray as xr
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from dask.diagnostics import ProgressBar

# Loading in AWAP

In [6]:
directory  = '/g/data/w40/ab2313/'

In [2]:
path = directory + 'precip_calib_0.25_1911_2017_land.nc'
AWAP = xr.open_dataset(path, chunks={'time':1}) 

In [4]:
AWAP.attrs = {}

# Applying  a Mask

In [7]:
#This file is a mask for AWAP: The Gibson Desert Region

path = directory  + 'precip_calib_0.25_maskforCAus.nc'
mask = xr.open_dataset(path)
mask = mask.rename({'longitude':'lon'})
mask = mask.rename({'latitude':'lat'})

AWAP = AWAP.where(mask.mask == 1)

# Just North and Raindays

In [11]:
 # Just the north
AWAP = AWAP.sel(lat = slice(-23,0))
#Rainday > 1mm
AWAP = AWAP.where(AWAP.precip >= 1, drop = True) 
 # This are unphysical
AWAP = AWAP.where(AWAP.precip < 8000, drop = True)
# wet season only
AWAP = AWAP.where(AWAP.time.dt.month.isin([10,11,12,1,2,3]) , drop = True) 

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in greater_equal
  return func(*args2)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in less
  return func(*args2)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in greater_equal
  return func(*args2)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in less
  return func(*args2)


In [15]:
AWAP.attrs = ({'Information':'Only contains the wet season [10,11,12,1,2,3],'
               + 'rainfall >= 1mm and the North of Australia'})

In [16]:
AWAP

<xarray.Dataset>
Dimensions:  (lat: 49, lon: 149, time: 19281)
Coordinates:
  * lon      (lon) float64 113.8 114.0 114.2 114.5 ... 150.0 150.2 150.5 150.8
  * lat      (lat) float64 -23.0 -22.75 -22.5 -22.25 ... -11.5 -11.25 -11.0
  * time     (time) datetime64[ns] 1911-01-01T09:00:00 ... 2017-12-31T09:00:00
Data variables:
    precip   (time, lat, lon) float32 dask.array<chunksize=(1, 49, 149), meta=np.ndarray>
Attributes:
    Information:  Only contains the wet season [10,11,12,1,2,3],rainfall >= 1...

In [17]:
with ProgressBar():
    AWAP.to_netcdf('/g/data/w40/ab2313/awap_n2.nc')

[                                        ] | 0% Completed | 22.2s

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in greater_equal
  return func(*args2)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in less
  return func(*args2)


[########################################] | 100% Completed | 14min 46.7s
